In [ ]:
import numpy as np
import os
import pandas as pd
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import openpyxl
import matplotlib as mpl
import scipy as sp
from scipy import stats
from scipy import signal
from scipy import fft
from scipy import io
from scipy import interpolate
from scipy import optimize
from scipy import linalg
from scipy import integrate
from scipy.fft import dct
from scipy import signal

mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = [
    r'\usepackage{textcomp}',
    r'\usepackage{wasysym}']
mpl.rcParams['mathtext.fontset'] = 'stix'
mpl.rcParams['font.size'] = 22
mpl.rcParams['font.family'] = 'STIXGeneral'
from BackDiffuse_LT import BackDiffuse


showFigs=True

In [ ]:
import h5py
filename = '/home/thea/MesterTesen/CFM-thea/cfm_mytests/steady_state_test_results_temp/SiteA_test.hdf5' 

f = h5py.File(filename,'r')
for key in f.keys():
    print(key)
    
age = f['age'][()]
depth = f['depth'][()]
temperature = f['temperature'][()]
Modelclimate = f['Modelclimate'][()]
density = f['density'][()]
#isotopes= f['isotopes'][()]
iso_sigma18 = f['iso_sigma18'][()]
iso_dsigma2_dt_18 = f['iso_dsigma2_dt_18'][()]
drho_dt = f['drho_dt'][()]

In [ ]:
fig, ax = plt.subplots()
ax.plot(depth[-1][1:],drho_dt[-1][1:],'.-')
#ax.set(ylim=(243,244.5))

In [ ]:
ys_d18O_forcing_SiteA = np.hstack(np.array([np.arange(1891,1920),np.arange(1931,1960)]))-1985
d18O_forcing_SiteA = np.hstack(np.array([-34.02*np.ones(1920-1891),-32.86*np.ones(1960-1931)]))

d18O_forcing_SiteA_Full = np.asarray([ys_d18O_forcing_SiteA,d18O_forcing_SiteA])

In [ ]:


ys_accum_forcing_SiteA = np.arange(1943,1973) - 1985
accum_forcing_SiteA = 0.307 * np.ones(1973-1943)

accum_forcing_SiteA_Full = np.asarray([ys_accum_forcing_SiteA,accum_forcing_SiteA])

In [ ]:
ys_temp_forcing_SiteA = np.arange(1948,1985)-1985
temp_forcing_SiteA = 29.41 * np.ones(1985-1948)+273.15

temp_forcing_SiteA_Full = np.asarray([ys_temp_forcing_SiteA,temp_forcing_SiteA])

In [ ]:
np.savetxt('/home/thea/MesterTesen/CFM-thea/cfm_mytests/cfm_my_forcings/temp_forcing_SiteA_TEST.csv',temp_forcing_SiteA_Full,delimiter=',')
np.savetxt('/home/thea/MesterTesen//CFM-thea/cfm_mytests/cfm_my_forcings/accum_forcing_SiteA_TEST.csv',accum_forcing_SiteA_Full,delimiter=',')
np.savetxt('/home/thea/MesterTesen/CFM-thea/cfm_mytests/cfm_my_forcings/d18O_forcing_SiteA_TEST.csv',d18O_forcing_SiteA_Full,delimiter=',')

In [ ]:
d18O_forcing_SiteA_Full

In [ ]:
test = pd.read_csv('/home/thea/MesterTesen/CFM-thea/cfm_mytests/cfm_my_forcings/accum_forcing_SiteA_TEST2.csv',',',header=None).transpose()

In [ ]:
np.loadtxt('/home/thea/MesterTesen/CFM-thea/cfm_mytests/cfm_my_forcings/temp_forcing_SiteA_TEST.csv',delimiter=',')